In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
#current 
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers


# dimensions of our images.
img_width, img_height = 256, 256

train_data_dir = '/kaggle/input/batch 8 40x giemsa combined/train/'
validation_data_dir = '/kaggle/input/batch 8 40x giemsa combined/validation/'
nb_train_samples = 700
nb_validation_samples = 175
epochs = 500
batch_size =8


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(64, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(128, (3, 3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))

model.add(Conv2D(256, (3, 3),input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(3, 3)))



model.add(Flatten())
model.add(Dense(32))
model.add(Activation('sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(1))
model.add(Activation('sigmoid'))

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.8, nesterov=True)
model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    #rotation_range=20,
    zoom_range=0.2,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
    #fill_mode="nearest"
)

validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    #rotation_range=20,
    zoom_range=0.2,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
    #fill_mode="nearest"
)


# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
from keras.utils import np_utils
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import numpy as np
from imutils import paths
from keras.preprocessing.image import ImageDataGenerator


totalTest = len(list(paths.list_images('/kaggle/input/batch 8 40x giemsa combined/test/')))

test_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    #rotation_range=20,
    zoom_range=0.2,
    #width_shift_range=0.1,
    #height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
    #fill_mode="nearest"
)



In [ ]:
img_width, img_height = 256, 256
batch_size =8
test_data_dir = '/kaggle/input/batch 8 40x giemsa combined/test/'
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')



In [ ]:
#print(len(list(test_generator)))
predIdxs = model.predict_generator(test_generator,steps=(totalTest // 8) + 1)
 
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
#print(predIdxs)
 
# show a nicely formatted classification report
print(test_generator.classes)
print(predIdxs)
print(classification_report(test_generator.classes, predIdxs,target_names=test_generator.class_indices.keys()))
cm = confusion_matrix(test_generator.classes, predIdxs)
print(cm)